# RS-UCB-M for K-arm

Import necessary libraries

In [1]:
import numpy as np

Define the utility function <br />  
The parameter of the utility function is the weighted average of the means

In [2]:
def Mixtures(tauO, p_arms):
    #tau0 = tau(T)/T
    #p_arms --> bernoulli probability of each arm
    p_out = 0
    for i in range(len(tauO)):
        p_out += tauO[i]*p_arms[i]
    return p_out

In [3]:
def Utility(p):
    #utility function
    return p*(1-p)

Define the confidence set function

In [4]:
def phi_inverse(x, v):
    #phi inverse function for confidence ellipsoids
    return 2*np.sqrt(x/v)

Choose an estimator at time instant t. IF the estimator chosen at time instant t-1, still meet the conditions choose that one. Other wise choose another estimator that meets the conditions randomly.

In [5]:
def UCBalpha(I, CS, K, pre, init=True):  
    '''
    Parameters:
    I --> epsilon list for possible alpha values
    CS --> confidence sets (K, 2) dimensional
    pre --> alpha value chosen at the previous time instant
    
    Outputs:
    opt_alpha --> K-dimension sampling probability estimator
    
    '''
    opt_alpha_SET = [] #only alpha scalar values
    opt_alpha_SET_actual = [] #alpha vectors
    arm_set = []
    opt_alpha = np.zeros((K))
    temp_alpha = np.zeros((K))
    
    non_zero_indices = np.nonzero(pre)[0]
    
    
    for idx, alphaT in enumerate(I):
        C_temp = np.zeros((2))
        
        temp = np.zeros((2))
        
        for i in range(K):
            for j in range(i+1, K): 
                '''
                only add the alpha values that can create a confidence set with a range that can 
                have 1/2 in it
                '''
                temp[0] = alphaT*CS[i][0]+(1-alphaT)*CS[j][0]
                temp[1] = alphaT*CS[i][1]+(1-alphaT)*CS[j][1]
      
                if temp[0] <= 1/2 <= temp[1]:
                    temp_alpha = np.zeros((K))
                    opt_alpha_SET.append(alphaT)
                    arm_set.append([i, j])
                    temp_alpha[i] = alphaT
                    temp_alpha[j] = 1-alphaT

                    opt_alpha_SET_actual.append(temp_alpha)
                    

                else:
                    pass
            
    #if the alpha vector chosen at the previous time instant 
    #was added to be possibly chosen then choose that alpha again
    if np.any(np.all(pre == np.array(opt_alpha_SET_actual), axis=1)):
        opt_alpha = pre

       

    else: 
        random_idx = np.random.randint(len(opt_alpha_SET))

        opt_alpha_value = opt_alpha_SET[random_idx]

        opt_alpha[arm_set[random_idx][0]] = opt_alpha_value

        opt_alpha[arm_set[random_idx][1]] = 1-opt_alpha_value
        
        
    return opt_alpha 

Run the main algorithm function <br />  
First run the exploration stage. Then, do under-sampling.

In [6]:
def UCBK(zeta=0.05, K=5, T = 100000, pb = [0.35, 0.85, 0.4, 0.7, 0.01], v=2, L=1, q=1):
    '''
    Parameters:
    
    Output:
    '''
    eps = np.sqrt(K/T)*np.sqrt(np.log(T))
    I = np.linspace(0, 1, int(np.ceil(1/eps)))

    tau_T = np.zeros((K))

    #pb = [p1, p2, p3, p4, p5]

    emp_pb = np.zeros((K))
    
    CS = np.zeros((K, 2))
    
    regrets = []
    
    opt_alpha = np.zeros((K))
    
    #EXPLORATION
    starttime = 0
    for i in range(K):
        for time in range(starttime, starttime + int(zeta*T)):
            reward = np.random.binomial(size=1, n=1, p=(pb[i]))
            
            emp_pb[i] = float(emp_pb[i]*tau_T[i] + reward)/(tau_T[i] + 1)
            
            tau_T[i] += 1
            
    
        starttime = time + 1
        
        
    #CREATING CONFIDENCE SETS AFTER EXPLORATION   
    for i in range(K):    
        CS[i][1] = min(emp_pb[i] + phi_inverse(np.log(T)/tau_T[i], v), 1)
        CS[i][0] = max(emp_pb[i] - phi_inverse(np.log(T)/tau_T[i], v), 0)
        
    opt_alpha = UCBalpha(I, CS, K, pre=opt_alpha)
    
    

    #UCB
    for time in range(starttime, T):
        #undersampling
        chosen_arm = np.argmax(opt_alpha - tau_T/time)
        
        reward = np.random.binomial(size=1, n=1, p=(pb[chosen_arm]))
        
        emp_pb[chosen_arm] = float(emp_pb[chosen_arm]*tau_T[chosen_arm] + reward)/(tau_T[chosen_arm] + 1)

        tau_T[chosen_arm] += 1
        
        #update the confidence sets
        for i in range(K):
            CS[i][1] = min(emp_pb[i] + phi_inverse(np.log(T)/tau_T[i], v), 1)
            CS[i][0] = max(emp_pb[i] - phi_inverse(np.log(T)/tau_T[i], v), 0)
        
        #choose an alpha for the next time instant
        opt_alpha = UCBalpha(I, CS, K, pre = opt_alpha)
        

    #calculate the simple regret
    regret = 1/4-Utility(Mixtures(tau_T/T, pb))
        
    return regret, emp_pb, tau_T, CS

    

## Number of Arms Experiments


K_list := list of different number of arms <br />
file_path := name of the txt file for data to be saved <br />
p1 := mean of the first arm <br />
p2 := mean of the second arm <br />
zeta := exploration coefficient <br />

In [7]:
T = 300000
K_list = [2, 3, 4, 5, 6]
pb = [0.45, 0.60, 0.75, 0.9, 0.3 , 0.15]

file_path = "Regrets_K.txt"
for K in K_list:
    for save_idx in range(1000):
        regret, emp_pb, tau_T, CS = UCBK(zeta=0.05, K=K, T = T, pb=pb[:K], v=2)
        

        with open(file_path, "a+") as file:
            file.write(f'T={T}, zeta={0.05}, K={K}, pb={pb}, RESULT: tau(T)={tau_T}, emp_pb={emp_pb} \n')

## Exploration Coefficient Experiments


zeta_list := list of different number of arms <br />
file_path := name of the txt file for data to be saved <br />
pb := list of means of the arms <br />
T := time horizon <br />

In [9]:
T = 75000
K = 3
pb = [0.1, 0.7, 0.4]

zeta_list = [0.01, 0.03, 0.05, 0.075, 0.1, 0.2]

file_path = "Regrets_zeta.txt"
for zeta in zeta_list:
    for save_idx in range(1000):
        regret, emp_pb, tau_T, CS = UCBK(zeta=zeta, K=K, T = T, pb=pb, v=2, L=1)
        

        with open(file_path, "a+") as file:
            file.write(f'T={T}, zeta={zeta}, K={K}, pb={pb}, RESULT: tau(T)={tau_T}, emp_pb={emp_pb} \n')